In [2]:
import pandas as pd
import numpy as np
import re
import time
import requests as rq
import bs4 as bs4

import json
import os 
import tqdm
import glob

In [2]:
# Setup the year and maker I want
makers = ["ford" , "vw-volkswagen"]
year_1 = "31" # 2013
year_2 = "36" #2018


In [8]:
url = "https://rj.olx.com.br/autos-e-pecas/carros-vans-e-utilitarios/{maker}/flex?o={page}&re={year_2}&rs={year_1}"

In [9]:
# User- agent for get response
headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.149 Safari/537.36'}

# Get data

In [11]:
for maker in makers:
    for page in range(1,101):
        urll = url.format(maker = maker, page = page, year_2 = year_2, year_1 = year_1)
        print(urll)
        
        # Get request -  to request data from the server.
        response = rq.get(urll, headers= headers)
        
        if response == 403:
            print('403 error - Forbidden! Please identify user-agent')
        else:
            with open('./raw_data/{}_{}.html'.format(maker,page), 'w+') as output:
                output.write(response.text)
            time.sleep(2)
        

https://rj.olx.com.br/autos-e-pecas/carros-vans-e-utilitarios/ford/flex?o=1&re=36&rs=31
https://rj.olx.com.br/autos-e-pecas/carros-vans-e-utilitarios/ford/flex?o=2&re=36&rs=31
https://rj.olx.com.br/autos-e-pecas/carros-vans-e-utilitarios/ford/flex?o=3&re=36&rs=31
https://rj.olx.com.br/autos-e-pecas/carros-vans-e-utilitarios/ford/flex?o=4&re=36&rs=31
https://rj.olx.com.br/autos-e-pecas/carros-vans-e-utilitarios/ford/flex?o=5&re=36&rs=31
https://rj.olx.com.br/autos-e-pecas/carros-vans-e-utilitarios/ford/flex?o=6&re=36&rs=31
https://rj.olx.com.br/autos-e-pecas/carros-vans-e-utilitarios/ford/flex?o=7&re=36&rs=31
https://rj.olx.com.br/autos-e-pecas/carros-vans-e-utilitarios/ford/flex?o=8&re=36&rs=31
https://rj.olx.com.br/autos-e-pecas/carros-vans-e-utilitarios/ford/flex?o=9&re=36&rs=31
https://rj.olx.com.br/autos-e-pecas/carros-vans-e-utilitarios/ford/flex?o=10&re=36&rs=31
https://rj.olx.com.br/autos-e-pecas/carros-vans-e-utilitarios/ford/flex?o=11&re=36&rs=31
https://rj.olx.com.br/autos-e-

# Raw data Preprocessing

In [70]:
for maker in makers:
    for page in range(1,101):
        with open('./raw_data/{}_{}.html'.format(maker, page ), 'r+') as inp:
            page_html = inp.read()
            
            parsed = bs4.BeautifulSoup(page_html)
            
            tags = parsed.findAll("a")
            
            for e in tags:
                if e.has_attr('data-lurker_list_id'):
                    link = e['href']
                    title = e['title']
                    with open("parsed_links.json", "a+") as output:
                        data = {"link":link,
                               "title":title,
                               "maker":maker}
                        output.write("{}\n".format(json.dumps(data)))
   

# DataFrame

In [8]:
df = pd.read_json("parsed_links.json", lines = True)
df

,link,title,maker
0,https://rj.olx.com.br/rio-de-janeiro-e-regiao/autos-e-pecas/carros-vans-e-utilitarios/ford-ka-2017-2018-1-0-ti-vct-se-12v-flex-4p-manual-731931843,FORD KA 2017/2018 1.0 TI-VCT SE 12V FLEX 4P MANUAL,ford
1,https://rj.olx.com.br/rio-de-janeiro-e-regiao/autos-e-pecas/carros-vans-e-utilitarios/ecosport-freestyle-1-6-2014-725699985,Ecosport freestyle 1.6 2014,ford
2,https://rj.olx.com.br/rio-de-janeiro-e-regiao/autos-e-pecas/carros-vans-e-utilitarios/ford-ka-hatch-1-0-completo-flex-2017-738467400,Ford KA Hatch 1.0 Completo Flex - 2017,ford
3,https://rj.olx.com.br/norte-do-estado-do-rio/autos-e-pecas/carros-vans-e-utilitarios/ecosport-2015-15-unico-dono-736481845,Ecosport 2015/15 único dono,ford
4,https://rj.olx.com.br/rio-de-janeiro-e-regiao/autos-e-pecas/carros-vans-e-utilitarios/ford-ecosport-freestyle-1-6-financiamos-sem-comprovacao-de-renda-730555720,Ford-Ecosport freestyle 1.6 Financiamos sem comprovação de renda,ford
...,...,...,...
6869,https://rj.olx.com.br/norte-do-estado-do-rio/autos-e-pecas/carros-vans-e-utilitarios/volkswagen-saveiro-cross-cd-572058952,Volkswagen Saveiro Cross CD,vw-volkswagen
6870,https://rj.olx.com.br/rio-de-janeiro-e-regiao/autos-e-pecas/carros-vans-e-utilitarios/volkswagen-gol-1-0-tec-8v-flex-4p-manual-570220869,Volkswagen Gol 1.0 tec 8v flex 4p manual,vw-volkswagen
6871,https://rj.olx.com.br/rio-de-janeiro-e-regiao/autos-e-pecas/carros-vans-e-utilitarios/vw-gol-confortline-flex-2015-477688486,VW - Gol confortline Flex 2015,vw-volkswagen
6872,https://rj.olx.com.br/rio-de-janeiro-e-regiao/autos-e-pecas/carros-vans-e-utilitarios/volkswagen-fox-1-0-mi-comfortline-8v-flex-4p-manual-448663594,Volkswagen Fox 1.0 mi comfortline 8v flex 4p manual,vw-volkswagen


In [7]:
df.link

0                     https://rj.olx.com.br/rio-de-janeiro-e-regiao/autos-e-pecas/carros-vans-e-utilitarios/ford-ka-2017-2018-1-0-ti-vct-se-12v-flex-4p-manual-731931843
1                                            https://rj.olx.com.br/rio-de-janeiro-e-regiao/autos-e-pecas/carros-vans-e-utilitarios/ecosport-freestyle-1-6-2014-725699985
2                                   https://rj.olx.com.br/rio-de-janeiro-e-regiao/autos-e-pecas/carros-vans-e-utilitarios/ford-ka-hatch-1-0-completo-flex-2017-738467400
3                                             https://rj.olx.com.br/norte-do-estado-do-rio/autos-e-pecas/carros-vans-e-utilitarios/ecosport-2015-15-unico-dono-736481845
4       https://rj.olx.com.br/rio-de-janeiro-e-regiao/autos-e-pecas/carros-vans-e-utilitarios/ford-ecosport-freestyle-1-6-financiamos-sem-comprovacao-de-renda-730555720
                                                                                      ...                                                                  

In [6]:
pd.set_option('max_colwidth', None)